In [1]:
import pandas as pd
from pathlib import Path
import os
import plotly.graph_objects as go
import json


# Input Data

In [30]:
DATA_DIR = Path(os.path.abspath('')).parents[1] / "data"
FARE_PRICES_DIR = DATA_DIR / "fares"
STOCK_PRICES_DIR = DATA_DIR / "stocks"
STOCK_PRICES_DIR

PosixPath('/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks')

In [31]:
tflFares = pd.read_csv(FARE_PRICES_DIR /"TfLHistoricalFares2000to2025.csv")
tflFares

,year,singleZ1to4Cash,singleZ1to4OysterPeak,busCash,singleBusOyster,capBusTram,travelcardZ1to4,capZ1to4PAYG,travelcard7DayZ1to4,weeklyBusAndTramPass
0,2000,2.6,NaN,1.00,NaN,NaN,NaN,NaN,26.8,11.5
1,2001,2.7,NaN,1.00,NaN,NaN,NaN,NaN,27.6,9.5
2,2002,2.7,NaN,1.00,NaN,NaN,6.8,NaN,28.1,8.5
3,2003,2.8,NaN,1.00,NaN,NaN,7.0,NaN,28.4,8.5
4,2004,3.0,2.8,1.00,0.70,NaN,7.3,NaN,29.2,9.5
5,2005,2.8,2.5,1.20,1.00,3.00,8.0,NaN,30.4,11.0
6,2006,3.0,2.5,1.50,1.00,3.00,8.4,NaN,31.6,13.5
7,2007,4.0,2.5,2.00,1.00,3.00,9.0,NaN,33.2,14.0
8,2008,4.0,2.5,2.00,0.90,3.00,9.4,NaN,34.6,13.0
9,2009,4.0,2.8,2.00,1.00,3.30,10.0,NaN,36.8,13.8


In [32]:
railFares = pd.read_csv(FARE_PRICES_DIR / "railFares.csv", skiprows=7)
railFares
# The prices are broken down
# - yearly
# - quarterly
# - monthly

,Important notes,Unnamed: 1
0,1987,100.6
1,1988,107.6
2,1989,117.4
3,1990,127.7
4,1991,141.0
...,...,...
642,2024 SEP,515.0
643,2024 OCT,523.4
644,2024 NOV,519.5
645,2024 DEC,519.6


In [6]:
busFares = pd.read_csv(FARE_PRICES_DIR / "busAndCoachFares.csv", skiprows=7)
busFares
# Same breakdown as railFares

,Important notes,Unnamed: 1
0,1987,103.4
1,1988,110.6
2,1989,119.3
3,1990,125.9
4,1991,143.6
...,...,...
642,2024 SEP,634.7
643,2024 OCT,630.1
644,2024 NOV,631.2
645,2024 DEC,647.1


In [7]:
# Need the following
# - Real wages

# Charts

In [8]:
# UTILS

LINE_AND_MARKER = "lines+markers"

def create_trace(x, y, label: str, mode=LINE_AND_MARKER, line_shape=None) -> go.Scatter:
    return go.Scatter(x=x, y=y, mode=mode, name=label, line_shape=line_shape)

def create_layout(title: str, type=None):
    return dict(
        title=title,
        width = 960,
        height = 500,
        xaxis=dict(
            showgrid=False,
            linecolor="#7f7f7f",
            linewidth=2,
            ticks='outside',
            type=type
        ),
        showlegend=True,
        plot_bgcolor='white'
    )

def normalise_single_series_to_100(df):
    return (df - df.min())/(df.max()-df.min()) * 100

def normalise_series_to_first_value(df):
    return (df / df.iloc[0])

## Tube Fares

In [9]:
fig = go.Figure()
# TRACES
fig.add_trace(create_trace(x=tflFares['year'], y=tflFares['singleZ1to4OysterPeak'], label="Contactless ticket"))
fig.add_trace(create_trace(x=tflFares['year'], y=tflFares['singleZ1to4Cash'], label="Paper ticket"))
# LAYOUT
layout = create_layout(title="TfL Tube Fares <br><sup>Zone 1-4 travel during peak hours</sup>")
fig.update_layout(layout)

fig.show()

# TODO: add annotiations for starting and final prices
# Follow code here: https://plotly.com/python/line-charts/ -> section Label Lines with Annotations

# 2000 - 2.6, 2012 - 5.3 for paper and 3.6 for contactless, 2025 - 7 for paper and 4.6 for contactless -> +38.46%, +76.92% using contactless
# Average Weekly Earnings (AWE) comparison: 2000 - £305, 2012 - £457, 2025 - £711 -> 49.83%, 133.11%
# https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/timeseries/kab9/emp
# Median weekly earnings for full-time employees: 2012 - £506, 2025 - £728
# https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/bulletins/annualsurveyofhoursandearnings/2024

In [10]:
# tflFares["singleZ1to4OysterPeak"][10:] / tflFares["singleZ1to4OysterPeak"][10:].iloc[0]
normalise_series_to_first_value(tflFares["singleZ1to4OysterPeak"][10:]) * 100 - 100

10     0.000000
11     9.677419
12    16.129032
13    22.580645
14    22.580645
15    25.806452
16    25.806452
17    25.806452
18    25.806452
19    25.806452
20    25.806452
21    29.032258
22    38.709677
23    41.935484
24    41.935484
25    48.387097
Name: singleZ1to4OysterPeak, dtype: float64

In [11]:
fig = go.Figure()
# TRACES
fig.add_trace(create_trace(x=tflFares['year'][10:], y=normalise_series_to_first_value(tflFares["singleZ1to4OysterPeak"][10:]) * 100 - 100, label="Contactless ticket"))
fig.add_trace(create_trace(x=tflFares['year'][10:], y=normalise_series_to_first_value(tflFares["singleZ1to4Cash"][10:]) * 100 - 100, label="Paper ticket"))
# fig.add_trace(create_trace(x=tflFares['year'][10:], y=normalise_single_series_to_100(tflFares["singleZ1to4OysterPeak"][10:]), label="Contactless ticket"))
# fig.add_trace(create_trace(x=tflFares['year'][10:], y=normalise_single_series_to_100(tflFares["singleZ1to4Cash"][10:]), label="Paper ticket"))
# LAYOUT
layout = create_layout(title="Normalised TfL Tube Fares <br><sup>Zone 1-4 travel during peak hours</sup>")
fig.update_layout(layout)

fig.show()

In [12]:
# Get pay-as-you-go caps as well
# How many journeys are required to reach the cap? -> could put it as barchart underneath
# tflFares['capZ1to4PAYG']/tflFares['singleZ1to4OysterPeak']

In [13]:
fig = go.Figure()
# TRACES
fig.add_trace(create_trace(x=tflFares['year'], y=tflFares['capZ1to4PAYG'], label="PAYG Cap"))
# fig.add_trace(create_trace(x=tflFares['year'], y=tflFares['travelcard7DayZ1to4'], label="travelcard7DayZ1to4"))
# LAYOUT
layout = create_layout(title="TfL Tube Fares <br><sup>Zone 1-4 travel during peak hours</sup>")
fig.update_layout(layout)

fig.update_xaxes(range=[2012,2025])

fig.show()

## FTSE100

In [15]:
data = json.load(open(STOCK_PRICES_DIR / "ftse100from2000to2025.json"))
ftse100 = pd.DataFrame(data['data'])
ftse100
# Month end value are reported from 2000 Jan to 2025 March + mid April

,_DATE_END,LOW_1,CLOSE_PRC,HIGH_1,OPEN_PRC
0,2000-01-31,6246.8,6268.54,6930.2,6930.2
1,2000-02-29,5972.7,6232.56,6450.9,6268.5
2,2000-03-31,6232.6,6540.22,6770.4,6232.6
3,2000-04-30,5915.2,6327.43,6586.3,6540.2
4,2000-05-31,5991.9,6359.35,6419.9,6327.4
...,...,...,...,...,...
299,2024-12-31,8002.34,8173.02,8388.37,8287.3
300,2025-01-31,8160.6,8673.96,8692.84,8173.02
301,2025-02-28,8520.2,8809.74,8820.93,8673.96
302,2025-03-31,8481.11,8582.81,8908.82,8809.74


In [16]:
ftse100['_DATE_END'] = pd.to_datetime(ftse100['_DATE_END'])
ftse100['year'] = ftse100['_DATE_END'].dt.year
# ftse100['year'] = ftse100['_DATE_END'].apply(lambda x: int(x.split('-')[0]))

ftse_yearly_max_prices = ftse100.groupby("year", as_index=False).max() # HALO this is stupid
ftse_yearly_max_prices = ftse_yearly_max_prices.sort_values("year")

ftse_yearly_min_prices = ftse100.groupby("year", as_index=False).min()
ftse_yearly_min_prices = ftse_yearly_min_prices.sort_values("year")

ftse_yearly_min_prices.head()

,year,_DATE_END,LOW_1,CLOSE_PRC,HIGH_1,OPEN_PRC
0,2000,2000-01-31,5915.2,6142.19,6419.9,6142.2
1,2001,2001-01-31,4219.8,4903.39,5279.8,4903.4
2,2002,2002-01-31,3609.9,3721.75,4197.5,3721.8
3,2003,2003-01-31,3277.5,3567.41,3747.0,3567.4
4,2004,2004-01-31,4283.0,4385.67,4487.9,4385.7


In [17]:
fig = go.Figure()
# TRACES
fig.add_trace(create_trace(x=ftse_yearly_max_prices['year'][10:], y=ftse_yearly_max_prices['HIGH_1'].astype(float)[10:], label="Highest annual price"))
fig.add_trace(create_trace(x=ftse_yearly_min_prices['year'][10:], y=ftse_yearly_min_prices['LOW_1'].astype(float)[10:], label="Lowest annual price"))
# LAYOUT
layout = create_layout(title="FTSE100<br><sup>UK's stock market's index innit</sup>", type="linear")
fig.update_layout(layout)

fig.show()

In [18]:
fig = go.Figure()
# TRACES
fig.add_trace(create_trace(x=ftse_yearly_max_prices['year'][10:], y=normalise_single_series_to_100(ftse_yearly_max_prices['LOW_1'].astype(float)[10:]), label="Lowest annual price"))
# LAYOUT
layout = create_layout(title="Normalised FTSE100<br><sup>UK's stock market's index innit</sup>", type="linear")
fig.update_layout(layout)

# fig.update_xaxes(range=[2010,2025])

fig.show()

In [21]:
# Could factor in inflation using CPIH
# CPIH stands for the Consumer Prices Index including owner occupiers' housing costs, which is the UK's leading measure of inflation
# Source: https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/l55o/mm23
cpih = pd.read_csv(DATA_DIR / "CPIH-annual-rate-1989-2024-by-year.csv", skiprows=7)
n_years = 2024 - 1989 + 1
cpih.iloc[:n_years].tail()

,key,annual_rate
31,2020,1.0
32,2021,2.5
33,2022,7.9
34,2023,6.8
35,2024,3.3


## Combined Charts

In [27]:
fig = go.Figure()

# TODO - identifier multiplier dynamically
fares_normalised = normalise_series_to_first_value(tflFares["singleZ1to4OysterPeak"][10:])
stocks_normalised = normalise_series_to_first_value(ftse_yearly_max_prices['HIGH_1'].astype(float)[10:])

# TRACES
fig.add_trace(create_trace(x=ftse_yearly_max_prices['year'][10:], y=stocks_normalised * 200 - 200, label="FTSE100 highs"))
fig.add_trace(create_trace(x=tflFares['year'][10:], y=fares_normalised * 200 - 200, label="Contactless ticket"))
# fig.add_trace(create_trace(x=tflFares['year'][10:], y=normalise_series_to_first_value(tflFares["singleZ1to4Cash"][10:]) * 200 - 200, label="Paper ticket"))
# fig.add_trace(create_trace(x=ftse_yearly_min_prices['year'][10:], y=normalise_series_to_first_value(ftse_yearly_max_prices['LOW_1'].astype(float)[10:]) * 200 - 200, label="FTSE100 lows"))

# LAYOUT
layout = create_layout(title="TfL Tube Fares vs the FTSE100<br><sup>Prices are normalised between 0-100</sup>")
fig.update_layout(layout)
fig.update_yaxes(title_text="Relative price")
fig.update_xaxes(title_text="Year")

fig.show()

In [25]:
layout

{'title': 'TfL Tube Fares vs the FTSE100<br><sup>Prices are normalised between 0-100</sup>',
 'width': 960,
 'height': 500,
 'xaxis': {'showgrid': False,
  'linecolor': '#7f7f7f',
  'linewidth': 2,
  'ticks': 'outside',
  'type': None},
 'showlegend': True,
 'plot_bgcolor': 'white'}

In [ ]:
m1 = fares_normalised.max()
m2 = stocks_normalised.max()
m = max(m1, m2)
print(m)

stocks_normalised

96.77419354838707


10     0.000000
11     2.800318
12    -0.811099
13    28.370528
14    29.341721
15    36.578504
16    37.245784
17    55.672877
18    62.511085
19    56.664995
20    55.408821
21    47.685445
22    55.329106
23    67.279364
24    81.473596
25    95.902323
Name: HIGH_1, dtype: float64

In [ ]:
normalise_single_series_to_100(pd.concat([tflFares["singleZ1to4OysterPeak"][10:], ftse_yearly_max_prices['HIGH_1'].astype(float)[10:]], axis=1))
# I need the same starting point and for the max to be 100

,singleZ1to4OysterPeak,HIGH_1
10,0.000000,0.838662
11,20.000000,3.734142
12,33.333333,0.000000
13,46.666667,30.173296
14,46.666667,31.177493
15,53.333333,38.660201
16,53.333333,39.350157
17,53.333333,58.403451
18,53.333333,65.474040
19,53.333333,59.429284


In [ ]:
# Create step non-decreasing step function
fig = go.Figure()

fares_normalised = normalise_series_to_first_value(tflFares["singleZ1to4OysterPeak"][10:]) * 200 - 200
stocks_normalised = normalise_series_to_first_value(ftse_yearly_max_prices['HIGH_1'].astype(float)[10:]) * 200 - 200
# TRACES
fig.add_trace(create_trace(
    x=tflFares['year'][10:],
    y=fares_normalised, 
    label="Contactless ticket (PAYG)",
    line_shape="hv"
))
fig.add_trace(create_trace(
    x=ftse_yearly_max_prices['year'][10:], 
    y=stocks_normalised, 
    label="FTSE100 annual highs",
    line_shape="hv"
))

# LAYOUT
layout = create_layout(title="TfL Tube Fares vs the FTSE100<br><sup>Prices are normalised between 0-100</sup>")
fig.update_layout(layout)

fig.show()

In [ ]:
((fares_normalised - stocks_normalised) > 0).sum()
# 11 out of 15 years the tube prices rose above the FTSE100

# Reselling your tube tickets would reap in more revenue than investing it in the UK economy
#  through a stock ISA. Sadiq, Rachel - wanna chip in?

np.int64(11)

In [ ]:
# Create plotly/streamlit chart where one can select the following from a dropdown
# - Stock Index -> Get S&P 500
# Tranportation mode -> we have average rail and bus/coach fares too from 

# Also need a dropdown for selecting years (2000-2025) and deal with missing values pls
# Though fares go back to 1987, and so do most indeces -> Did a search in perplexity

# Sources
# - Bus & coach: https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/docx/mm23
# - Train: https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/docw/mm23
# - FTSE100: downloaded from WSJ Markets
# - S&P 500: downloaded from WSJ Markets
# - DAX: downloaded from WSJ Markets
# - Nikkei 225: downloaded from WSJ Markets
# - Hang Seng: downloaded from WSJ Markets